In [1]:
import hmac
import hashlib

# 双方协商出的共享 MAC 密钥（通常由 Master Secret 派生）
mac_key = b'shared_mac_key_32bytes_longgggggggggg'

# 客户端要发送的消息
message = b'hello, this is a TLS message'

# --- 客户端生成 MAC ---
def generate_hmac(key, message):
    return hmac.new(key, message, hashlib.sha256).digest()

client_hmac = generate_hmac(mac_key, message)
print("客户端生成的 HMAC:", client_hmac.hex())

# --- 模拟消息被服务端接收 ---
# 服务端接收到 message 和 client_hmac

# --- 服务端验证 HMAC ---
def verify_hmac(key, message, received_hmac):
    expected_hmac = hmac.new(key, message, hashlib.sha256).digest()
    return hmac.compare_digest(expected_hmac, received_hmac)

# 验证是否被篡改
is_valid = verify_hmac(mac_key, message, client_hmac)
print("消息是否未被篡改:", is_valid)

# --- 模拟篡改消息 ---
tampered_message = b'hello, this is a TLS messAge'  # 篡改了一个字母

is_valid_tampered = verify_hmac(mac_key, tampered_message, client_hmac)
print("篡改后的消息是否未被篡改:", is_valid_tampered)


客户端生成的 HMAC: 51083ccc7ce9c2ae695087063b8fa6e10c786beca7379f60838c0d900a3e8e63
消息是否未被篡改: True
篡改后的消息是否未被篡改: False


In [2]:
import os
import hmac
import hashlib
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives import padding

# 1. DH 密钥交换
parameters = dh.generate_parameters(generator=2, key_size=2048)
client_private = parameters.generate_private_key()
server_private = parameters.generate_private_key()
client_public = client_private.public_key()
server_public = server_private.public_key()
shared_secret = client_private.exchange(server_public)
assert shared_secret == server_private.exchange(client_public)

# 2. 派生主密钥 -> 加密密钥 & MAC 密钥（简单示例用 SHA256 hash 拆分）
master_secret = hashlib.sha256(shared_secret).digest()
encryption_key = master_secret[:16]  # AES-128
mac_key = master_secret[16:]        # 余下作为 HMAC key

# 3. 客户端准备发送消息
plaintext = b"This is a confidential TLS message."

# 3.1 计算 MAC（先计算 MAC，再加密）
mac = hmac.new(mac_key, plaintext, hashlib.sha256).digest()

# 3.2 拼接消息 + MAC
message_with_mac = plaintext + mac

# 3.3 AES-CBC 加密
iv = os.urandom(16)
padder = padding.PKCS7(128).padder()
padded_data = padder.update(message_with_mac) + padder.finalize()
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv))
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded_data) + encryptor.finalize()

print("[Client] 发送加密消息和 IV")

# ------------------------ 服务端接收并处理 ------------------------

# 4. 服务端接收 ciphertext 和 IV，解密
cipher = Cipher(algorithms.AES(encryption_key), modes.CBC(iv))
decryptor = cipher.decryptor()
decrypted_padded = decryptor.update(ciphertext) + decryptor.finalize()

# 4.1 去填充
unpadder = padding.PKCS7(128).unpadder()
decrypted = unpadder.update(decrypted_padded) + unpadder.finalize()

# 4.2 拆出原文与 MAC
recv_plaintext = decrypted[:-32]
recv_mac = decrypted[-32:]

# 4.3 重新计算 MAC 验证完整性
expected_mac = hmac.new(mac_key, recv_plaintext, hashlib.sha256).digest()
is_valid = hmac.compare_digest(recv_mac, expected_mac)

# 输出结果
print("\n[Server] 解密后明文:", recv_plaintext.decode())
print("[Server] 校验 MAC 是否一致:", is_valid)


[Client] 发送加密消息和 IV

[Server] 解密后明文: This is a confidential TLS message.
[Server] 校验 MAC 是否一致: True
